# Broadcast

### But : Faire un broadcast, départ décalé 


#### Informations générales

In [1]:
experiment_name = "Bcast"

In [2]:
%%bash -s
git log -n 1
python --version

commit a04f47f8581ba06caf6cbe4ae6309b6f7c02f6aa
Author: Najwa Ez Zine <nezzine@fnancy>
Date:   Thu May 24 15:30:08 2018 +0200

    Notebook update


Python 2.7.13


#### Préparation de l'environnement

In [3]:
%%bash -s
pip3 install --user execo

In [4]:
import execo
from execo import *
from execo_g5k import *
from execo_engine import *
import os
import sys
import json
import time
import datetime
import re
from shutil import copy
from subprocess import check_output

##### Réservation noeuds

In [5]:
# Nom du job
jobname = ' '
# Nombre de noeuds
nodecount = 5
# Temps réservation
walltime = "6:0:0" 

# filters out Nancy's graphene cluster (buggy)
resources_selection = "-p \"cluster != 'graphene'\"" 

# Nancy
site = "nancy"

In [6]:
jobs = get_current_oar_jobs()
jobid = None
waiting_jobs = []
while jobs:
    j, site = jobs.pop()
    info = get_oar_job_info(j, site)
    if info['name'] == jobname:
        if info['state'] == 'Running':
            jobid = j
            print("A {} job is already running, using it. jobid is {}".format(jobname, jobid))
            break
        else:
            waiting_jobs.append(j)
if not jobid and not waiting_jobs:
    jobspec = OarSubmission(resources="/cluster=1/nodes={}".format(nodecount), walltime=walltime,
                            additional_options=resources_selection, job_type="deploy", name=jobname)
    jobid, _ = oarsub([(jobspec, site)]).pop()
    print("New job submitted, jobid is {}".format(jobid))
elif not jobid:
    print("One or more {} jobs exist ({}) but are not running.\n"
          " Connect to the frontend to see what is happening, and/or run the cell again.".format(
          jobname, ", ".join([str(j) for j in waiting_jobs])))

New job submitted, jobid is 1572171


On les range

In [7]:
nodes = get_oar_job_nodes(jobid)
nodes.sort(key=lambda n: n.address)
nodes

[Host('grisou-3.nancy.grid5000.fr'),
 Host('grisou-40.nancy.grid5000.fr'),
 Host('grisou-43.nancy.grid5000.fr'),
 Host('grisou-49.nancy.grid5000.fr'),
 Host('grisou-50.nancy.grid5000.fr')]

##### Déploiement de l'environnement

In [8]:
force_redeploy = False # set to True to force redeploying the OS on the nodes in the deployment section
environment_dsc_file = '../../../../../EP/debian9-x64-bigdata-tutorial.yaml' # filename of the kadeploy environment file (YAML)

In [9]:
deployment = Deployment(hosts=nodes, env_file = os.path.abspath(environment_dsc_file),
                        other_options="-r ext4 --no-debug-mode")

deploy_ok, deploy_failed = deploy(deployment, check_deployed_command=not force_redeploy,
                              stdout_handlers=[sys.stdout],
                              stderr_handlers=[sys.stderr])

nancy: Deployment #D-7bb4de8a-2f3c-4142-be01-0a16b6c8684e started
nancy: Grab the key file /home/nezzine/.ssh/authorized_keys
nancy: Grab the tarball file http://public.nancy.grid5000.fr/~pneyron/debian9-x64-bigdata-tutorial.tar.gz
nancy: Grab the postinstall file server:///grid5000/postinstalls/g5k-postinstall.tgz
nancy: Launching a deployment on grisou-[3,40,43,49-50].nancy.grid5000.fr
nancy: Performing a Deploy[SetDeploymentEnvUntrusted] step
nancy:   switch_pxe
nancy:   reboot
nancy:    * Performing a soft reboot on grisou-[3,40,43,49-50].nancy.grid5000.fr
nancy:   wait_reboot
nancy:   send_key_in_deploy_env
nancy:   create_partition_table
nancy:   format_deploy_part
nancy:   mount_deploy_part
nancy:   format_tmp_part
nancy:   format_swap_part
nancy: End of step Deploy[SetDeploymentEnvUntrusted] after 177s
nancy: Performing a Deploy[BroadcastEnvKascade] step
nancy:   send_environment
nancy:    * Broadcast time: 70s
nancy:   manage_admin_post_install
nancy:   manage_user_post_instal

nancy: Warning: Permanently added 'nancy.grid5000.fr,172.16.79.101' (ECDSA) to the list of known hosts.
nancy: Connection to nancy.grid5000.fr closed.


#### Test : commande simple 'ls'

In [120]:
# execo.action.Remote(cmd,hosts,connection_params,process_args)
Remote_test = execo.action.Remote(cmd='ls',hosts=nodes,connection_params=None, process_args=None)
Remote_test.run().ok

Remote('ls', [Host('grisou-3.nancy.grid5000.fr'), Host('grisou-40.nancy.grid5000.fr'), Host('grisou-43.nancy.grid5000.fr'), Host('grisou-49.nancy.grid5000.fr'), Host('grisou-50.nancy.grid5000.fr')])

#### Installation environnement sur noeud

In [11]:
Remote_install = execo.action.Remote(cmd='apt-get install g++ libboost-all-dev && wget http://gforge.inria.fr/frs/download.php/latestfile/8/SimGrid-3.18.tar.gz && tar -xvf SimGrid-3.18.tar.gz && cd SimGrid-3.18 && cmake -DCMAKE_INSTALL_PREFIX=/usr/local -Denable_smpi=on -Denable_documentation=off && make -j && make check && make install -j',hosts=nodes,connection_params={'user':'root'})

In [12]:
Remote_install.run().ok

True

In [134]:
execo.action.Remote(cmd='touch smpi_version.txt && smpicc --version > smpi_version.txt',hosts=nodes,connection_params={'user':'root'}).run()
version = ''
for i in range(0,nodecount) :
    execo.action.Get(hosts=nodes[i],remote_files=["~/smpi_version.txt"], local_location='./smpicc_version-'+get_host_shortname(nodes[i])+'.txt',connection_params={'user':'root'}).run()
    version = version + execo.Process(cmd='cat ./smpicc_version-'+get_host_shortname(nodes[i])+'.txt').run().stdout.replace('\n','  ')
    execo.Process(cmd='rm ./smpicc_version-'+get_host_shortname(nodes[i])+'.txt').run()
version

'SimGrid version 3.18  SimGrid version 3.18  SimGrid version 3.18  SimGrid version 3.18  SimGrid version 3.18  '

In [13]:
path = "~/Stage-POLARIS/SimGrid/examples/SMPI/"+experiment_name+'/src/'

#### Récupération fichiers sources

In [33]:
Src_get = execo.action.Remote(cmd='mkdir -p experiments_src',hosts=nodes,connection_params={'user':'root'})
Src_get.run().ok

In [34]:
Remote_smpi_env = execo.action.Put(hosts=nodes,local_files=[path+"griffon.xml",path+"griffon_hostfile.txt",path+"broadcast.c",path+"Makefile"],remote_location='~/experiments_src',connection_params={'user':'root'})
Remote_smpi_env.run().ok

#### Execution de l'expérience

In [35]:
Remote_smpi = execo.action.Remote(cmd ='cd ~/experiments_src && make && make run NPROCS=2 MIN=0 MAX=10',hosts=nodes,connection_params={'user':'root'})
Remote_smpi.run().ok

#### Récupération des résultats

In [36]:
Result_get = execo.Process(cmd='mkdir -p /home/nezzine/Stage-POLARIS/SimGrid/examples/SMPI/'+experiment_name+'/results/'+datetime.datetime.now().strftime("%Y-%m-%d"))
Result_get.run().ok

##### Shortcut

In [102]:
Src_get = execo.action.Remote(cmd='mkdir -p experiments_src',hosts=nodes,connection_params={'user':'root'})
Remote_smpi_env = execo.action.Put(hosts=nodes,local_files=[path+"griffon.xml",path+"griffon_hostfile.txt",path+"broadcast.c",path+"Makefile"],remote_location='~/experiments_src',connection_params={'user':'root'})

def parl_exec () :
    NPROCS = 20
    N = 12
    for i in range (0,nodecount) :
       print("i = "+str(i)+" NPROCS = "+str(NPROCS)+" MIN = "+str((int)((i*N)/len(nodes)))+" MAX = "+str((int)(((i+1)*N)/len(nodes)))+" N="+str(N))
       bool = execo.action.Remote(cmd ='cd ~/experiments_src && make && make run NPROCS='+str(NPROCS)+' MIN='+str((int)((i*N)/len(nodes))) +' MAX='+str((((i+1)*N)/len(nodes)))+' N='+str(N),hosts=nodes[i],connection_params={'user':'root'}).run().ok
       print(bool)
    return;
# Remote_smpi = execo.action.Remote(cmd ='cd ~/experiments_src && make && make run NPROCS=2 MIN=0 MAX=10',hosts=nodes,connection_params={'user':'root'})
Result_get = execo.Process(cmd='mkdir -p /home/nezzine/Stage-POLARIS/SimGrid/examples/SMPI/'+experiment_name+'/results/'+datetime.datetime.now().strftime("%Y-%m-%d"))

Src_get.run().ok
Remote_smpi_env.run().ok
#Remote_smpi.run().ok
parl_exec()
Result_get.run().ok

i = 0 NPROCS = 20 MIN = 0 MAX = 2 N=12
True
i = 1 NPROCS = 20 MIN = 2 MAX = 4 N=12
True
i = 2 NPROCS = 20 MIN = 4 MAX = 7 N=12
True
i = 3 NPROCS = 20 MIN = 7 MAX = 9 N=12
True
i = 4 NPROCS = 20 MIN = 9 MAX = 12 N=12
True


True

In [103]:
for i in range(0,nodecount) :
    execo.action.Get(hosts=nodes[i],remote_files=["~/experiments_src/bcast_results.txt"], local_location='~/Stage-POLARIS/SimGrid/examples/SMPI/'+experiment_name+'/results/'+datetime.datetime.now().strftime("%Y-%m-%d")+'/'+get_host_shortname(nodes[i])+'_'+datetime.datetime.now().strftime("%Y-%m-%d_%H:%M")+'.txt',connection_params={'user':'root'}).run().ok

#### Publication des résultats

In [104]:
%%bash -s $experiment_name
git add .
git commit -m 'Automatic results update : '"$1"

[master fd44d09] Automatic results update : Bcast
 Committer: Najwa Ez Zine <nezzine@fnancy>
Your name and email address were configured automatically based
on your username and hostname. Please check that they are accurate.
You can suppress this message by setting them explicitly. Run the
following command and follow the instructions in your editor to edit
your configuration file:

    git config --global --edit

After doing this, you may fix the identity used for this commit with:

    git commit --amend --reset-author

 7 files changed, 830 insertions(+), 137 deletions(-)
 create mode 100644 SimGrid/examples/SMPI/Bcast/results/2018-05-25/grisou-3_2018-05-25_12:11.txt
 create mode 100644 SimGrid/examples/SMPI/Bcast/results/2018-05-25/grisou-40_2018-05-25_12:11.txt
 create mode 100644 SimGrid/examples/SMPI/Bcast/results/2018-05-25/grisou-43_2018-05-25_12:11.txt
 create mode 100644 SimGrid/examples/SMPI/Bcast/results/2018-05-25/grisou-49_2018-05-25_12:11.txt
 create mode 100644 SimGrid

#### Fin d'expérience : suppression du job

In [ ]:
oardel(jobid)